In [353]:
import torch
import torch.nn as nn

In [364]:
#Separated the Identity block and conv block for better understanding, We can modularize it if required
# @Santosh

class IdentityBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(IdentityBlock, self).__init__()
        self.conv1 = nn.Conv2d(
                               in_channels=in_channels,
                               out_channels=out_channels,
                               kernel_size=1,
                               stride=1, 
                               padding=0,
                               bias=False
        )
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(
                               in_channels=out_channels,
                               out_channels=out_channels,
                               kernel_size=3,
                               stride=stride,
                               padding=1,
                               bias=False
        )
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(
                              in_channels=out_channels,
                              out_channels=out_channels*4,
                              kernel_size=1,
                              stride=1,
                              padding=0,
                              bias=False
        )
        self.batch_norm3 = nn.BatchNorm2d(out_channels*4)
        self.relu = nn.ReLU()
        
    def forward(self, x):
        identity = x.clone()
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.batch_norm3(x)
        x += identity
        x = self.relu(x)
        return x

In [366]:
class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ConvBlock, self).__init__()
        self.conv1 = nn.Conv2d(
                               in_channels=in_channels,
                               out_channels=out_channels,
                               kernel_size=1,
                               stride=1, 
                               padding=0,
                               bias=False
        )
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(
                               in_channels=out_channels,
                               out_channels=out_channels,
                               kernel_size=3,
                               stride=stride,
                               padding=1,
                               bias=False
        )
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(
                              in_channels=out_channels,
                              out_channels=out_channels*4,
                              kernel_size=1,
                              stride=1,
                              padding=0,
                              bias=False
        )
        self.batch_norm3 = nn.BatchNorm2d(out_channels*4)
        
        self.conv4 = nn.Conv2d(
                              in_channels=in_channels,
                              out_channels=out_channels*4,
                              kernel_size=1,
                              stride=stride,
                              bias=False
        )
        self.batch_norm4 = nn.BatchNorm2d(out_channels*4)
        
        self.relu = nn.ReLU()
        
    def forward(self, x):
        identity = x.clone()
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.batch_norm2(x)
        x = self.relu(x)
        x = self.conv3(x)
        x = self.batch_norm3(x)
        
        down_sample = self.conv4(identity)
        down_sample = self.batch_norm4(down_sample)       
            
        x += down_sample
        x = self.relu(x)
        return x

In [370]:
class ResNet(nn.Module):
    def __init__(self, layers=None, num_classes=None):
        super(ResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(
            in_channels=3,
            out_channels=64,
            kernel_size=7,
            stride=2,
            padding=3,
            bias=False
        )
        
        self.batch = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=(3,3), stride=2, padding=1)
        
        self.layer1 = self.resnet_blocks(64, layers[0], 1) 
        self.layer2 = self.resnet_blocks(128, layers[1], 2)
        self.layer3 = self.resnet_blocks(256, layers[2], 2)
        self.layer4 = self.resnet_blocks(512, layers[3], 2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * 4, num_classes)
        
        
    def resnet_blocks(self, out_channels, num_blocks, stride=1):
        layers =  []

        layers.append(
            ConvBlock(self.in_channels, out_channels, stride)
        )
        
        self.in_channels = out_channels * 4
    
        for _ in range(num_blocks-1):
            layers.append(IdentityBlock(self.in_channels, out_channels))
            
        return nn.Sequential(*layers)    
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.batch(x)
        x = self.relu(x)
        x = self.pool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc(x)
        
        return x
        


In [371]:
def ResNet50(num_classes=1000):
    return Resnet([3, 4, 6, 3], num_classes)

def ResNet101(img_channel=3, num_classes=1000):
    return ResNet([3, 4, 23, 3], num_classes)


def ResNet152(num_classes=1000):
    return ResNet([3, 8, 36, 3], num_classes)

In [374]:
image = torch.randn(1,3,224,224)

model = ResNet50(1000)
model(image).shape

torch.Size([1, 1000])

In [375]:
#Total trainable paratmeters
sum([p.numel() for p in model.parameters() if p.requires_grad])

25557032